# Working With Data 

Import all of our packages 

In [164]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import nltk 
import plotly as py
import plotly.graph_objs as go
from functools import reduce
py.offline.init_notebook_mode(connected=True)
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\brsegal\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [102]:
loans = pd.read_csv('kiva_loans.csv')
mpi_regions = pd.read_csv('kiva_mpi_region_locations.csv')
MPI_subnational = pd.read_csv('MPI_subnational.csv')
MPI_national = pd.read_csv('MPI_national.csv')

In [188]:
uses = loans['use'].value_counts().head(2000)

In [104]:
data = [go.Bar(x=uses.index,
            y=uses.values)]
py.offline.offline.iplot(data)

In [180]:
def get_term(document):
    def normalise(word):
        """Normalises words to lowercase and stems and lemmatizes it."""
        word = word.lower()
        # word = stemmer.stem_word(word) #if we consider stemmer then results comes with stemmed word, but in this case word will not match with comment
        word = lemmatizer.lemmatize(word)
        return word
    def leaves(tree):
        """Finds NP (nounphrase) leaf nodes of a chunk tree."""
        for subtree in tree.subtrees(filter = lambda t: t.label()=='NP'):
            yield subtree.leaves()
    
    lemmatizer = nltk.WordNetLemmatizer()
    stemmer = nltk.stem.porter.PorterStemmer()
    
    sentences = nltk.sent_tokenize(document)
    sentences = [nltk.word_tokenize(sent) for sent in sentences] 
    sentences = [nltk.pos_tag(sent) for sent in sentences]
    sentences = reduce(lambda x,y: x+y,sentences)
    grammar = r"""
        NBAR:
            {<NN.*|JJ>*<NN.*>}  # Nouns and Adjectives, terminated with Nouns

        NP:
            {<NBAR>}
            {<NBAR><IN><NBAR>}  # Above, connected with in/of/etc...
    """
    cp = nltk.RegexpParser(grammar) 
    tree = cp.parse(sentences)
    for index,leaf in enumerate(leaves(tree)):
        term = ' '.join([ normalise(w) for w,t in leaf])
        if index==0:
            return(term)
        
    

In [193]:
uses_nouns = {}
uses_values= uses.values.tolist()
uses_names = uses.index.tolist()
print(uses_list)
for i,use in enumerate(uses_names):
    noun = get_term(use)
    if noun in uses_nouns:
        uses_nouns[noun]+=uses_values[i]
    else:
        uses_nouns[noun]=uses_values[i]
uses_nouns

[5217, 4082, 2141, 1708, 1599, 1419, 1409, 1405, 1280, 1261, 1154, 1027, 970, 888, 881, 877, 866, 843, 785, 760, 747, 688, 681, 667, 621, 604, 583, 579, 577, 572, 568, 562, 559, 547, 527, 501, 487, 486, 467, 456, 452, 450, 432, 432, 430, 420, 420, 419, 413, 406, 405, 398, 393, 389, 388, 385, 384, 381, 376, 375, 374, 373, 372, 365, 359, 357, 350, 345, 344, 343, 331, 325, 320, 320, 311, 309, 306, 304, 296, 286, 280, 278, 274, 274, 273, 272, 269, 265, 264, 262, 261, 259, 259, 258, 256, 254, 245, 244, 244, 237, 233, 232, 231, 230, 227, 225, 223, 219, 215, 215, 214, 214, 213, 212, 211, 210, 207, 207, 199, 199, 198, 197, 197, 197, 195, 193, 192, 191, 190, 190, 189, 188, 188, 187, 186, 186, 186, 185, 185, 184, 183, 183, 180, 179, 177, 176, 176, 173, 170, 170, 170, 170, 169, 168, 168, 167, 167, 167, 166, 165, 165, 165, 162, 162, 160, 160, 158, 158, 158, 157, 157, 155, 155, 154, 153, 152, 152, 152, 151, 151, 150, 150, 150, 149, 147, 146, 146, 145, 145, 145, 145, 144, 144, 144, 142, 141, 141, 14

{None: 1425,
 'access': 45,
 'access high quality input': 168,
 'access high-quality input': 655,
 'additional box': 1079,
 'additional container': 117,
 'additional fish': 176,
 'additional fishing net': 59,
 'additional fresh fish': 51,
 'additional good': 37,
 'additional grocery': 354,
 'additional grocery good': 285,
 'additional grocery item': 58,
 'additional household good': 45,
 'additional item': 625,
 'additional livestock': 35,
 'additional material': 112,
 'additional nipa': 41,
 'additional piglet': 35,
 'additional poultry': 97,
 'additional prepaid airtime credit': 142,
 'additional sack': 1658,
 'additional spare part': 61,
 'additional stock': 2007,
 'additional vegetable': 57,
 'agricultural input': 55,
 'agricultural supply': 193,
 'agriculture input': 41,
 'airtime': 42,
 'animal': 368,
 'animal feed': 130,
 'assorted drink': 36,
 'baby goat': 215,
 'baby poultry': 366,
 'bag': 589,
 'bale': 674,
 'bamboo stick': 36,
 'banana': 84,
 'beauty product': 41,
 'beehive'

In [231]:
uses_export = pd.DataFrame(list(uses_nouns.items()),columns=['Loans Use','Loans Count'])
uses_export = uses_export.set_index('Loans Use')
uses_export = uses_export.sort_values('Loans Count',ascending=False)
uses_export.to_csv('Loan Count by Use.csv')